In [24]:
import os
import sys
%load_ext autoreload
%autoreload 2
import pandas as pd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from vadbp.variant_comparator import VariantComparator, DataType
from vadbp.preprocessing import prepare_dataframe_edges

In [26]:
def clean_df(df):
    explode_df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
    explode_df['time:timestamp'] = explode_df[['intime', 'outtime']].values.tolist()
    explode_df['a'] = 'start'
    explode_df['b'] = 'complete'
    explode_df['lifecycle:transition'] = explode_df[['a', 'b']].values.tolist()
    explode_df = explode_df.explode(['time:timestamp', 'lifecycle:transition'])
    explode_df.drop(['intime', 'outtime', 'a', 'b'], axis=1 ,inplace=True)
    explode_df['concept:name'] = explode_df['department']
    explode_df['case:concept:name'] = explode_df['subject_id']
    return explode_df

In [27]:
csv_data = pd.read_csv('../Event Logs_2nd_Iteration/Heart_Failure_Log.csv')
csv_data = clean_df(csv_data)
csv_data

,subject_id,hadm_id,transfer_id,department,Glucose,Abnormal Glucose,Potassium,Abnormal Potassium,Sodium,Abnormal Sodium,...,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,time:timestamp,lifecycle:transition,concept:name,case:concept:name
0,10559183,20008400.0,31433513,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 13:32:00,start,Emergency Department,10559183
0,10559183,20008400.0,31433513,Emergency Department,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 16:00:00,complete,Emergency Department,10559183
1,10559183,20008400.0,36107959,Cardiac ICU,164.000000,1.0,4.316667,1.0,134.833333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-03 16:00:00,start,Cardiac ICU,10559183
1,10559183,20008400.0,36107959,Cardiac ICU,164.000000,1.0,4.316667,1.0,134.833333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-06 18:44:57,complete,Cardiac ICU,10559183
2,10559183,20008400.0,38390977,Post-ICU Cardiology,125.000000,1.0,4.300000,2.0,137.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2116-01-06 18:44:57,start,Post-ICU Cardiology,10559183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,11282384,29994261.0,30592303,Surgical ICU,138.833333,1.0,4.000000,2.0,142.333333,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-21 18:35:27,complete,Surgical ICU,11282384
4321,11282384,29994261.0,37585480,Post-ICU Medicine,114.333333,1.0,4.466667,2.0,142.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-21 18:35:27,start,Post-ICU Medicine,11282384
4321,11282384,29994261.0,37585480,Post-ICU Medicine,114.333333,1.0,4.466667,2.0,142.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-24 13:13:59,complete,Post-ICU Medicine,11282384
4322,11282384,29994261.0,39587256,Discharged,123.000000,1.0,4.300000,2.0,140.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2124-09-24 13:13:59,start,Discharged,11282384


In [28]:
# dataset split
subject_over_60  = csv_data[csv_data['age']>60]['subject_id']
subject_under_60  = csv_data[csv_data['age']<=60]['subject_id']
over_60 = csv_data[csv_data['subject_id'].isin(subject_over_60)]
under_60 = csv_data[csv_data['subject_id'].isin(subject_under_60)]

In [29]:
varcop = VariantComparator(over_60, under_60, csv_data)

/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/pm4py/objects/log/util/dataframe_utils.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], utc=True, infer_datetime_format=True)


In [30]:
# continuous vars
varcop.do_activity_comparison(DataType.CONTINUOUS)

In [8]:
# categorical vars
varcop.do_activity_comparison(DataType.CATEGORICAL)


/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/pingouin/contingency.py:151: UserWarning: Low count on observed frequencies.
  warnings.warn('Low count on {} frequencies.'.format(name))
/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/pingouin/contingency.py:151: UserWarning: Low count on expected frequencies.
  warnings.warn('Low count on {} frequencies.'.format(name))
/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/scipy/stats/stats.py:6716: RuntimeWarning: divide by zero encountered in power
  terms = f_obs * ((f_obs / f_exp)**lambda_ - 1)
/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/scipy/stats/stats.py:6716: RuntimeWarning: invalid value encountered in multiply
  terms = f_obs * ((f_obs / f_exp)**lambda_ - 1)
/opt/homebrew/Caskroom/miniforge/base/envs/newenv4/lib/python3.8/site-packages/scipy/stats/stats.py:6713: RuntimeWarning: divide by zero encountered i

In [8]:
varcop.significant_differences_for_activity()

p-val
concept:name         variable                       
Surgical ICU         DBP                4.155474e-06
                     Temperature F      9.413181e-04
Pre-ICU Cardiology   Urea Nitrogen      1.526282e-03
                     Magnesium          2.381893e-03
Post-ICU Medicine    pH                 1.946342e-03
                     PT                 8.042031e-04
                     INR(PT)            7.906411e-04
                     Phosphate          1.073295e-04
                     Abnormal INR(PT)   4.362156e-04
Post-ICU Cardiology  MCV                6.793420e-05
                     Red Blood Cells    3.305883e-04
Medical ICU          DBP                1.700985e-06
                     Temperature F      3.854910e-06
                     Respiratory Rate   1.156665e-05
                     PT                 8.966515e-05
                     INR(PT)            3.283458e-04
Emergency Department age               7.088673e-101
                     dbp                3.094515e-04
Discharged           Phosphate          4.192497e-08
                     Sodium             1.893308e-06
                     MCV                5.032724e-05
                     Chloride           1.016561e-04
Cardiac ICU          DBP                6.724052e-15
                     Urea Nitrogen      3.045820e-06
                     MCV                4.864896e-06
                     Temperature F      3.861002e-05
                     MCH                3.686980e-04

In [9]:
%matplotlib tk
plt, fig, gviz = varcop.visualize()

gviz_json = json.loads(gviz.pipe('json'))

x_max = plt.xlim()[1]
y_max = plt.ylim()[0]

node_bboxes = []

_, _, gviz_x_max, gviz_y_max = gviz_json['bb'].split(',')
gviz_x_max = float(gviz_x_max)
gviz_y_max = float(gviz_y_max)

for obj in gviz_json['objects']:
    if len(obj['_draw_']) == 3:
        node_bboxes.append((obj['label'], obj['_draw_'][2]['points']))

        
def onclick(event):
    evt_gviz_x = (gviz_x_max / x_max) * event.xdata
    evt_gviz_y = gviz_y_max - (gviz_y_max / y_max) * event.ydata
    
    print('gviz_x=%d, gviz_y=%d' % (evt_gviz_x, evt_gviz_y))
    for label, bb in node_bboxes:
        if bb[1][0] <= evt_gviz_x and evt_gviz_x <= bb[0][0] and bb[2][1] <= evt_gviz_y and evt_gviz_y <= bb[0][1]:
            print('ich will nicht in', label)
            
print(plt.xlim())
print(plt.ylim())
cid = plt.gcf().canvas.mpl_connect('button_press_event', onclick)

plt.show()

used attributes: ALL
(-0.5, 1221.5)
(592.5, -0.5)


In [10]:
#gviz.render(filename='g1.dot')

In [12]:
#json.loads(gviz.pipe('json'))

In [9]:
varcop.do_edge_comparison()

/Users/hendrik/dev/studium/data-based-process-variant-analysis/vadbp/preprocessing.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_side_df['variable'] = left_side_df['variable'].apply(lambda row: row.rstrip('_l'))
/Users/hendrik/dev/studium/data-based-process-variant-analysis/vadbp/preprocessing.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_side_df['variable'] = right_side_df['variable'].apply(lambda row: row.rstrip('_r'))
/Users/hendrik/dev/studium/data-based-process-variant-analysis

In [10]:
varcop.get_edge_comparison_results()

CLES_1  \
concept:name_l      concept:name_r      variable                               
Cardiac ICU         Post-ICU Cardiology Glucose                     0.606854   
                                        Hematocrit                  0.470948   
                                        White Blood Cells           0.594082   
                                        Calcium                     0.399598   
                                        Chloride                    0.563235   
                                        Bicarbonate                 0.370675   
                                        Abnormal Calcium            0.392857   
Post-ICU Cardiology Discharged          Abnormal Phosphate          0.349091   
Post-ICU Medicine   Discharged          Abnormal Phosphate          0.359589   
                                        Abnormal Bicarbonate        0.425258   
                                        Abnormal Chloride           0.405128   
                                        Abnormal Glucose            0.391089   
Post-ICU Cardiology Discharged          Abnormal White Blood Cells  0.440063   
                                        Abnormal Calcium            0.415129   
                                        Abnormal Bicarbonate        0.390490   
                                        Abnormal Magnesium          0.462733   
                                        Abnormal Anion Gap          0.432277   
                                        Abnormal Creatinine         0.470085   
                                        Abnormal Chloride           0.424501   
                                        Abnormal Sodium             0.424716   
                                        Abnormal Glucose            0.378223   
Post-ICU Medicine   Discharged          Abnormal Calcium            0.393103   

                                                                       RBC_1  \
concept:name_l      concept:name_r      variable                               
Cardiac ICU         Post-ICU Cardiology Glucose                     0.437982   
                                        Hematocrit                 -0.285472   
                                        White Blood Cells           0.543143   
                                        Calcium                    -0.434259   
                                        Chloride                    0.391729   
                                        Bicarbonate                -0.708747   
                                        Abnormal Calcium           -0.588235   
Post-ICU Cardiology Discharged          Abnormal Phosphate         -1.000000   
Post-ICU Medicine   Discharged          Abnormal Phosphate         -1.000000   
                                        Abnormal Bicarbonate       -1.000000   
                                        Abnormal Chloride          -1.000000   
                                        Abnormal Glucose           -1.000000   
Post-ICU Cardiology Discharged          Abnormal White Blood Cells -1.000000   
                                        Abnormal Calcium           -1.000000   
                                        Abnormal Bicarbonate       -1.000000   
                                        Abnormal Magnesium         -1.000000   
                                        Abnormal Anion Gap         -1.000000   
                                        Abnormal Creatinine        -1.000000   
                                        Abnormal Chloride          -1.000000   
                                        Abnormal Sodium            -1.000000   
                                        Abnormal Glucose           -1.000000   
Post-ICU Medicine   Discharged          Abnormal Calcium           -1.000000   

                                                                    RBC_abs_1  \
concept:name_l      concept:name_r      variable                                
Cardiac ICU         Post-ICU Cardiology Glucose                   

In [1]:

import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

fig, ax = plt.subplots()
ax.plot(np.random.rand(10))

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=236, y=190, xdata=4.233871, ydata=0.428858
single click: button=1, x=269, y=208, xdata=5.112097, ydata=0.481015
single click: button=1, x=277, y=223, xdata=5.325000, ydata=0.524479
single click: button=1, x=90, y=278, xdata=0.348387, ydata=0.683847
single click: button=1, x=392, y=88, xdata=8.385484, ydata=0.133301
single click: button=1, x=357, y=136, xdata=7.454032, ydata=0.272387
single click: button=1, x=310, y=142, xdata=6.203226, ydata=0.289772
single click: button=1, x=256, y=137, xdata=4.766129, ydata=0.275284
single click: button=1, x=248, y=143

In [ ]:
import graphviz
import vadbp.gviz_viz as viz


# create a simple graph
dot = graphviz.Digraph(comment='The Round Table', format='svg')
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')
dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')


# display the graph
server = viz.start_graph_server(
    gviz,
    # the context menu for when a node is clicked
    node_menu={
        'Option 1': lambda node: print("option 1,", node, "clicked"),
        'Option 2': lambda node: print("option 2,", node, "clicked"),
    },
    # the context menu for when an edge is clicked
    edge_menu={
        "Edge Context Item": lambda edge: print("edge,", edge, "clicked"),
        "Another Edge Context Item": lambda edge: print("another,", edge, "clicked"),
        "Does nothing": lambda edge: None
    }
)

In [11]:
html1 = """<!DOCTYPE html>
                <html>
                <head>
                    <meta charset="utf-8"/>
                    <style>
                        
.menu {
    box-shadow: 0 4px 5px 3px rgba(0, 0, 0, 0.2);
    position: absolute;
    display: none;
    background-color: white;
}
.menu-options {
    list-style: none;
    padding: 10px 0;
}
.menu-option {
    font-weight: 500;
    font-size: 14px;
    padding: 10px 40px 10px 20px;
    cursor: pointer;
    white-space: nowrap;
}
.menu-option:hover {
    background: rgba(0, 0, 0, 0.2);
}
// polygon, ellipse,
svg, title, path, text {
   pointer-events: none;
}

g {
    pointer-events: all;
}

                    </style>
                </head>
                <body>
                    <div id="graph"><?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.50.0 (20211204.2007)
 -->
<!-- Pages: 1 -->
<svg width="388pt" height="116pt"
 viewBox="0.00 0.00 387.68 116.00" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 112)">
<polygon fill="white" stroke="transparent" points="-4,4 -4,-112 383.68,-112 383.68,4 -4,4"/>
<!-- A -->
<g id="node1" class="node" data-activity="KING">
<title  data-activity="KING">A</title>
<ellipse fill="none" stroke="black" cx="188.69" cy="-90" rx="53.89" ry="18"  data-activity="KING"/>
<text text-anchor="middle" x="188.69" y="-86.3" font-family="Times,serif" font-size="14.00"  data-activity="KING">King Arthur</text>
</g>
<!-- B -->
<g id="node2" class="node"  data-activity="Bedevere">
<title  data-activity="Bedevere">B</title>
<ellipse fill="none" stroke="black" cx="89.69" cy="-18" rx="89.88" ry="18"  data-activity="Bedevere"/>
<text text-anchor="middle" x="89.69" y="-14.3" font-family="Times,serif" font-size="14.00"  data-activity="Bedevere">Sir Bedevere the Wise</text>
</g>
<!-- A&#45;&gt;B -->
<g id="edge1" class="edge">
<title>A&#45;&gt;B</title>
<path fill="none" stroke="black" d="M166.72,-73.46C153.39,-64.04 136.15,-51.85 121.36,-41.39"/>
<polygon fill="black" stroke="black" points="123.17,-38.38 112.98,-35.47 119.13,-44.1 123.17,-38.38"/>
</g>
<!-- L -->
<g id="node3" class="node">
<title>L</title>
<ellipse fill="none" stroke="black" cx="288.69" cy="-18" rx="90.98" ry="18"/>
<text text-anchor="middle" x="288.69" y="-14.3" font-family="Times,serif" font-size="14.00">Sir Lancelot the Brave</text>
</g>
<!-- A&#45;&gt;L -->
<g id="edge2" class="edge">
<title>A&#45;&gt;L</title>
<path fill="none" stroke="black" d="M210.89,-73.46C224.35,-64.04 241.77,-51.85 256.71,-41.39"/>
<polygon fill="black" stroke="black" points="258.98,-44.07 265.17,-35.47 254.97,-38.33 258.98,-44.07"/>
</g>
<!-- B&#45;&gt;L -->
<g id="edge3" class="edge">
<title>B&#45;&gt;L</title>
<path fill="none" stroke="black" d="M179.48,-18C182.09,-18 184.7,-18 187.31,-18"/>
<polygon fill="black" stroke="black" points="187.39,-21.5 197.39,-18 187.39,-14.5 187.39,-21.5"/>
</g>
</g>
</svg>
</div>
                </body>
                </html>
        """

In [23]:
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox, Output
from ipyevents import Event 
from IPython.display import display
import numpy as np
import pandas as pd


h2 = HTML(html1)
o1 = HTML()

d2 = Event(source=h2, watched_events=['click'], watched_node_attributes=['data-activity'])

out = Output()

def handle_event(event):
    o1.value = (f"You clicked a {event}")
    out.clear_output()
    out.append_display_data(varcop.dataframe_for_graph_element('edge', 'Surgical ICU', 'Cardiac ICU', 'Post-ICU Cardiology'))

d2.on_dom_event(handle_event)

display(VBox([HBox([h2, o1]), out]))

In [20]:
varcop.dataframe_for_graph_element('edge', data_id='Surgical ICU', data_id_left='Cardiac ICU', data_id_right='Post-ICU Cardiology')

p-val_1  \
concept:name_l concept:name_r      variable                          
Cardiac ICU    Post-ICU Cardiology Glucose            8.770649e-11   
                                   Hematocrit         2.770268e-05   
                                   White Blood Cells  2.130070e-15   
                                   Calcium            3.477236e-10   
                                   Chloride           6.950956e-09   
                                   Bicarbonate        4.059451e-25   
                                   Abnormal Calcium   2.868312e-09   

                                                           p-val_2     RBC_1  \
concept:name_l concept:name_r      variable                                    
Cardiac ICU    Post-ICU Cardiology Glucose            5.197453e-09  0.437982   
                                   Hematocrit         5.076651e-05 -0.285472   
                                   White Blood Cells  1.450828e-09  0.543143   
                                   Calcium            7.119405e-07 -0.434259   
                                   Chloride           2.837536e-05  0.391729   
                                   Bicarbonate        9.890645e-08 -0.708747   
                                   Abnormal Calcium   4.237704e-04 -0.588235   

                                                         RBC_2  RBC_diff  
concept:name_l concept:name_r      variable                               
Cardiac ICU    Post-ICU Cardiology Glucose            0.721264  0.283282  
                                   Hematocrit        -0.512335 -0.226863  
                                   White Blood Cells  0.764773  0.221630  
                                   Calcium           -0.619425 -0.185166  
                                   Chloride           0.529260  0.137531  
                                   Bicarbonate       -0.658046  0.050701  
                                   Abnormal Calcium  -0.625000 -0.036765

In [17]:
varcop.edge_comparison_results.reset_index()[
    (varcop.edge_comparison_results.reset_index()['concept:name_l'] == 'Cardiac ICU') 
    and
    (varcop.edge_comparison_results.reset_index()['concept:name_r'] == 'Post-ICU Cardiology')
]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
varcop.edge_comparison_results.reset_index()[
    (varcop.edge_comparison_results.reset_index()['concept:name_l'] == 'Cardiac ICU') 
    &
    (varcop.edge_comparison_results.reset_index()['concept:name_r'] == 'Post-ICU Cardiology')
]

,concept:name_l,concept:name_r,variable,CLES_1,RBC_1,RBC_abs_1,p-val_1,value_l_1,value_r_1,count_l_1,...,RBC_2,RBC_abs_2,p-val_2,value_l_2,value_r_2,count_l_2,count_r_2,p-val-threshold_2,RBC_diff,RBC_diff_abs
0,Cardiac ICU,Post-ICU Cardiology,Glucose,0.606854,0.437982,0.437982,8.770649e-11,"[165.0, 130.0, 250.4, 186.0, 166.2, 151.0, 158...","[95.05555555555556, 139.25, 147.35714285714286...",304,...,0.721264,0.721264,5.197453e-09,"[110.11764705882356, 113.33333333333331, 232.5...","[126.33333333333331, nan, 199.75, nan, 89.5, 1...",90,90,0.000556,0.283282,0.283282
1,Cardiac ICU,Post-ICU Cardiology,Hematocrit,0.470948,-0.285472,0.285472,2.770268e-05,"[26.5, 40.2, 33.85, 23.157142857142865, 31.666...","[26.19545454545455, 38.25, 29.70000000000001, ...",304,...,-0.512335,0.512335,5.076651e-05,"[32.57142857142857, 37.93333333333333, 29.65, ...","[35.3, nan, 30.399999999999995, nan, 39.8, 35....",90,90,0.000556,-0.226863,0.226863
2,Cardiac ICU,Post-ICU Cardiology,White Blood Cells,0.594082,0.543143,0.543143,2.130070e-15,"[15.2, 11.65, 14.85, 13.333333333333336, 11.43...","[11.68181818181818, 8.8, 11.163636363636362, 1...",304,...,0.764773,0.764773,1.450828e-09,"[9.857142857142858, 8.841666666666667, 3.3, 9....","[10.8, nan, 2.8000000000000003, nan, 10.0, 10....",90,90,0.000556,0.221630,0.221630
3,Cardiac ICU,Post-ICU Cardiology,Calcium,0.399598,-0.434259,0.434259,3.477236e-10,"[7.4, 8.8, 8.466666666666667, 8.66666666666666...","[8.711764705882352, 8.975, 8.799999999999997, ...",304,...,-0.619425,0.619425,7.119405e-07,"[8.417647058823528, 8.905882352941179, 8.8, 8....","[9.25, nan, 9.275, nan, 9.4, 9.411111111111111...",90,90,0.000556,-0.185166,0.185166
4,Cardiac ICU,Post-ICU Cardiology,Chloride,0.563235,0.391729,0.391729,6.950956e-09,"[116.5, 95.5, 94.8, 93.0, 103.4, 99.5555555555...","[105.33333333333331, 96.0, 94.06666666666666, ...",304,...,0.529260,0.529260,2.837536e-05,"[93.52941176470588, 95.0, 99.5, 102.0, 104.0, ...","[95.66666666666669, nan, 96.75, nan, 100.5, 86...",90,90,0.000556,0.137531,0.137531
5,Cardiac ICU,Post-ICU Cardiology,Bicarbonate,0.370675,-0.708747,0.708747,4.059451e-25,"[14.0, 27.75, 27.0, 28.0, 30.85714285714285, 2...","[21.55555555555556, 27.5, 27.66666666666667, 2...",304,...,-0.658046,0.658046,9.890645e-08,"[29.35294117647059, 27.70588235294117, 24.0, 2...","[25.0, nan, 24.5, nan, 24.5, 29.11111111111111...",90,90,0.000556,0.050701,0.050701
6,Cardiac ICU,Post-ICU Cardiology,Abnormal Calcium,0.392857,-0.588235,0.588235,2.868312e-09,"[1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, ...","[1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, ...",304,...,-0.625000,0.625000,4.237704e-04,"[1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, ...","[2.0, nan, 2.0, nan, 2.0, 2.0, 1.0, 2.0, 2.0, ...",90,90,0.000556,-0.036765,0.036765


In [11]:
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox
from ipyevents import Event 
from IPython.display import display

In [12]:
l2 = HTML('''
    <button id=eeny class="success special">eeny</button>
    <button id=meeny class="caution">meeny</button>
    <button id=miny>miny</button>
    <button id=moe>moe</button>
''')

h2 = HTML('Event info')
d2 = Event(source=l2, watched_events=['click'])

def handle_event(event):
    h2.value = (f"You clicked a {event['target']['tagName']} with "
                f"ID {event['target']['id']} " 
                f"and classes: {event['target']['className']}")

d2.on_dom_event(handle_event)

display(l2, h2)


HTML(value='\n    <button id=eeny class="success special">eeny</button>\n    <button id=meeny class="caution">…

HTML(value='Event info')

In [11]:
l

Label(value='Click or type on me!', layout=Layout(border='2px solid red'))